In [ ]:
WLseries1001_1600.mat

In [ ]:
########### Load Packages ##############

from scipy.io import netcdf
import numpy as np   
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt  # Python standard library datetime  module
import numpy as np
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy.random import seed
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import pandas
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from PermutationImportance import sklearn_permutation_importance
from PyALE import ale
from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV
import scipy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
import scipy.io
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression

###### Set up the font sizes for matplotlib ######
FONT_SIZE = 14
BIG_FONT_SIZE = FONT_SIZE + 2
LARGE_FONT_SIZE = FONT_SIZE + 4
HUGE_FONT_SIZE = FONT_SIZE + 6
SMALL_FONT_SIZE = FONT_SIZE - 2
TINY_FONT_SIZE = FONT_SIZE - 4
TEENSIE_FONT_SIZE = FONT_SIZE - 6
font_sizes = {
    'teensie': TEENSIE_FONT_SIZE,
    'tiny': TINY_FONT_SIZE,
    'small': SMALL_FONT_SIZE,
    'normal': FONT_SIZE,
    'big': BIG_FONT_SIZE,
    'large': LARGE_FONT_SIZE,
    'huge': HUGE_FONT_SIZE,
}
plt.rc('font', size=FONT_SIZE)
plt.rc('axes', titlesize=FONT_SIZE)
plt.rc('axes', labelsize=FONT_SIZE)
plt.rc('xtick', labelsize=TINY_FONT_SIZE)
plt.rc('ytick', labelsize=TINY_FONT_SIZE)
plt.rc('legend', fontsize=FONT_SIZE)
plt.rc('figure', titlesize=BIG_FONT_SIZE)

###### Define plot importance metric #########
def plot_variable_importance(importance_obj, filename, multipass=True, relative=False, num_vars_to_plot=10, diagnostics=0):
    """Plots any variable importance method for a particular estimator
    :param importance_obj: ImportanceResult object returned by PermutationImportance
    :param filename: string to place the file into (including directory and '.png')
    :param multipxass: whether to plot multipass or singlepass results. Default to True
    :param relative: whether to plot the absolute value of the results or the results relative to the original. Defaults
        to plotting the absolute results
    :param num_vars_to_plot: number of top variables to actually plot (cause otherwise it won't fit)
    :param diagnostics: 0 for no printouts, 1 for all printouts, 2 for some printouts. defaults to 0
    """

    rankings = importance_obj.retrieve_multipass(
    ) if multipass else importance_obj.retrieve_singlepass()

    original_score = importance_obj.original_score

    try:
        len(original_score)
    except:
        bootstrapped = False
    else:
        bootstrapped = True

    if bootstrapped:
        original_score_mean = np.mean(original_score)
    else:
        original_score_mean = original_score

    # Sort by increasing rank
    sorted_var_names = list(rankings.keys())
    sorted_var_names.sort(key=lambda k: rankings[k][0])
    sorted_var_names = sorted_var_names[:min(num_vars_to_plot, len(rankings))]
    scores = [rankings[var][1] for var in sorted_var_names]

    colors_to_plot = [variable_to_color(var) for var in [
        "Original Score", ] + sorted_var_names]
    variable_names_to_plot = [" {}".format(
        var) for var in convert_vars_to_readable(["Original Score", ] + sorted_var_names)]

    if bootstrapped:
        if relative:
            scores_to_plot = np.array([original_score_mean, ] + [np.mean(score)
                                                                 for score in scores]) / original_score_mean
        else:
            scores_to_plot = np.array(
                [original_score_mean, ] + [np.mean(score) for score in scores])
        ci = np.array([np.abs(np.mean(score) - np.percentile(score, [0.025, 0.975]))
                       for score in np.r_[[original_score, ], scores]]).transpose()
    else:
        if relative:
            scores_to_plot = np.array(
                [original_score_mean, ] + scores) / original_score_mean
        else:
            scores_to_plot = np.array(
                [original_score_mean, ] + scores)
        ci = np.array([[0, 0]
                       for score in np.r_[[original_score, ], scores]]).transpose()

    metric = "Score"
    if importance_obj.method == "Permutation Importance":
        method = "%s Permutation Importance" % (
            "Multipass" if multipass else "Singlepass")
    else:
        method = importance_obj.method
    title = "%s\n%s" % (metric, method)

    # Actually make plot
    fig = plt.figure(figsize=(4, 3))
    if bootstrapped:
        plt.barh(np.arange(len(scores_to_plot)),
                 scores_to_plot, linewidth=1, edgecolor='black', color=colors_to_plot, xerr=ci, capsize=4, ecolor='grey', error_kw=dict(alpha=0.4))
    else:
        plt.barh(np.arange(len(scores_to_plot)),
                 scores_to_plot, linewidth=1, edgecolor='black', color=colors_to_plot)

    # Put the variable names _into_ the plot
    for i in range(len(variable_names_to_plot)):
        plt.text(0, i, variable_names_to_plot[i],
                 va="center", ha="left", size=font_sizes['teensie'])
    if relative:
        plt.axvline(1, linestyle=':', color='grey')
        plt.text(1, len(variable_names_to_plot) / 2, "original score = %0.3f" % original_score_mean,
                 va='center', ha='left', size=font_sizes['teensie'], rotation=270)
        plt.xlabel("Percent of Original Score")
        plt.xlim([0, 1.2])
    else:
        plt.axvline(original_score_mean, linestyle=':', color='grey')
        plt.text(original_score_mean, len(variable_names_to_plot) / 2, "original score",
                 va='center', ha='left', size=font_sizes['teensie'], rotation=270)
        plt.xlabel("Score")

    plt.ylabel("Predictor Permuted")
    plt.title(title)
    plt.yticks([])
    plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.3f'))

    # make the horizontal plot go with the highest value at the top
    plt.gca().invert_yaxis()

    print("Saving file to %s" % filename)
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    
    
# You can fill this in by using a dictionary with {var_name: legible_name}
def convert_vars_to_readable(variables_list):
    """Substitutes out variable names for human-readable ones
    :param variables_list: a list of variable names
    :returns: a copy of the list with human-readable names
    """
    human_readable_list = list()
    for var in variables_list:
        if False:  # var in VARIABLE_NAMES_DICTONARY:
            pass  # human_readable_list.append(VARIABLE_NAMES_DICTONARY[var])
        else:
            human_readable_list.append(var)
    return human_readable_list


# This could easily be expanded with a dictionary
def variable_to_color(var):
    return "lightblue"





In [ ]:
##########################################
##########################################
############# Start NN deve ##############
######### Author JWLocwkwood  ############
######### Date last mod 10 June 2021  ####
##########################################

fname = "WATERLEVELS/saving_tracks/loc_200_all3_again/site_" + str(v) + "_tracks.mat" # File to read
x = scipy.io.loadmat(fname)
### Load input, output ###
levels2 = (x['levels22']) 
lon10022 = (x['lon10022'])
vstore22 = (x['vstore22'])
lat10022 = (x['lat10022'])
rostore22 = (x['rostore22'])
speed10022 = (x['speed10022'])
pstore10022 = (x['pstore10022'])
bearing10022 = (x['bearing10022'])

# Need to scale and df 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Fit and predict model 
history = best_mlp.fit(X_trains,y_trains)
y_pred = best_mlp.predict(X_tests)

####### LOOP through each tracks, scros for mse and then save MSE,RMSE,CORR,R2 and best parameters, and test train
for v in np.arange(1):
    vv = a1[v]
    v=vv
    fname = "/Data/r300_d300/site_" + str(v) + "_tracks.mat" # File to read
    x = scipy.io.loadmat(fname)
    disatnce22 = (x['disatnce22'])
    levels2 = (x['levels2'])
    lon10022 = (x['lon10022'])
    vstore22 = (x['vstore22'])
    lat10022 = (x['lat10022'])
    rostore22 = (x['rostore22'])
    rmw10022 = (x['rmw10022'])
    speed10022 = (x['speed10022'])
    disatnce22 = (x['disatnce22'])
    pstore10022 = (x['pstore10022'])
    bearing10022 = (x['bearing10022'])
    pstore1002grad2 = (x['pstore1002grad2'])
    vstore1002grad2 = (x['vstore1002grad2'])

    abc = (np.shape(levels2))
    abc = (abc[0])
    all_data = np.empty([8,abc])
    all_data[0,:] = bearing10022[:,0]
    all_data[1,:] = rmw10022[:,0]
    all_data[2,:] = speed10022[:,0]
    all_data[3,:] = vstore22[:,0]
    all_data[4,:] = rostore22[:,0]
    all_data[5,:] = pstore10022[:,0]
    all_data[6,:] = lon10022[:,0] 
    all_data[7,:] = lat10022[:,0] 

    y = np.asarray(levels2)
    X = np.asarray(all_data.T)
    y = np.asarray(levels2)
    X = np.asarray(all_data.T)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    from sklearn.preprocessing import StandardScaler
    scaler = MinMaxScaler()
    X_trains = scaler.fit_transform(X_train)
    scaler = MinMaxScaler()
    X_tests = scaler.fit_transform(X_test)
    scaler = MinMaxScaler()
    y_trains = scaler.fit_transform(y_train)
    scaler = MinMaxScaler()
    y_tests = scaler.fit_transform(y_test)

    estimator = MLPRegressor()
    param_grid = {'hidden_layer_sizes': [(100,200,100),(200, 200, 200),(200,100),(100,200,300),(200,300,300),(120,100)]}
    
    # Testing the hyperparameters #
    from sklearn.metrics import make_scorer
    mse = make_scorer(mean_squared_error,greater_is_better=False)
    gsc = TuneGridSearchCV(estimator, param_grid, scoring=mse, max_iters=5)
    grid_result = gsc.fit(X_trains, y_trains)
    best_params = grid_result.best_params_
    best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], activation = "relu", solver="adam", max_iter = 1000, n_iter_no_change = 400, batch_size = 100)
    history = best_mlp.fit(X_trains,y_trains)
    y_pred = best_mlp.predict(X_tests)
    
    
    